In [14]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import requests
import zipfile
from pyspark.sql import SparkSession

In [16]:
url = 'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2021.zip'

In [17]:
r = requests.get(url, verify=False)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [18]:
with open("r.zip", "wb") as f:
    f.write(r.content)

In [19]:
with zipfile.ZipFile("r.zip", "r") as zip_ref:
    zip_ref.extractall("./microData")

In [20]:
spark = SparkSession.builder.getOrCreate()

In [21]:
df = spark.read.csv("microData/Microdados do Censo da Educaç╞o Superior 2021/dados/MICRODADOS_CADASTRO_IES_2021.CSV", sep=';', header=True, inferSchema=True)

In [22]:
from pyspark.sql.functions import col

df = df.select([col(c).alias(c.lower()) for c in df.columns])

####  Olhando para o estado de Minas Gerais, quantos municípios têm informações presentes na base de dados?


In [23]:
df.createOrReplaceTempView('questao1')

df1 = spark.sql("""
    select COUNT(DISTINCT no_municipio_ies) from questao1
    where sg_uf_ies = 'MG'
""")
df1.show()

+--------------------------------+
|count(DISTINCT no_municipio_ies)|
+--------------------------------+
|                             104|
+--------------------------------+



#### Quantos professores doutores existem em cada cidade de Minas Gerais presente na base de dados 

In [24]:
df.createOrReplaceTempView('questao2')

df2 = spark.sql("""
    select no_municipio_ies nome_cidade, SUM(qt_doc_ex_dout) numero_doutores_por_cidade from questao2
    where sg_uf_ies = 'MG'
    group by no_municipio_ies
    order by no_municipio_ies
""")

df2.show()

+---------------+--------------------------+
|    nome_cidade|numero_doutores_por_cidade|
+---------------+--------------------------+
|        Aimor�s|                         5|
|        Alfenas|                       661|
|       Almenara|                         4|
|   Al�m Para�ba|                        10|
|       Araguari|                        55|
|          Arax�|                        24|
|       Baependi|                         1|
|      Barbacena|                        70|
|Bar�o de Cocais|                         0|
| Belo Horizonte|                      6615|
|     Bert�polis|                         2|
|          Betim|                       192|
|  Boa Esperan�a|                         1|
|   Bom Despacho|                       129|
|     Brumadinho|                         3|
|     Cambuquira|                         3|
|     Campo Belo|                         8|
|  Campos Gerais|                         8|
|      Capelinha|                         1|
|      Car

####  Qual a quantidade de docentes com deficiência no estado do Paraná?


In [25]:
df.createOrReplaceTempView('questao3')

df3 = spark.sql("""
    SELECT SUM(qt_doc_ex_com_deficiencia) AS pessoas_deficientes_parana from questao3
    where sg_uf_ies = 'PR'
""")

df3.show()

+--------------------------+
|pessoas_deficientes_parana|
+--------------------------+
|                       160|
+--------------------------+



#### Passando para parquet

In [27]:
df1.write.parquet('questao1')
df2.write.parquet('questao2')
df3.write.parquet('questao3')